## Attempting to parse larger trackman file to calculate run expectancies

In [9]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)

In [10]:
master_trackman = pd.read_csv('../Trackman CSV Files/merged_trackman_games.csv')
master_trackman.head()

/var/folders/bp/l1vb_64s6tj40wftrxyjhnfm0000gn/T/ipykernel_32849/1647163910.py:1: DtypeWarning: Columns (91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  master_trackman = pd.read_csv('../Trackman CSV Files/merged_trackman_games.csv')


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.11803,3.24925,-0.69951,50.0,5.66596,0.97194,-119.21935,2.16895,1.61728,21.82808,-27.47381,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashingto

## Missing aspects of the dataset:

- Current baserunner state
- Estimated runs scored

## How to go about it: 

- Initialize batters to bases based on the hit type
- Empty runners from the bases when inning turns over
- Take probability of runner taking an extra base from 2024 MLB data (probability of runner on second reaching home on a single)
- Use probability distribution based on MLB probabilities for extra base baserunner instances
- Handle rare instances like fielder's choice
- How do we handle stolen bases? 

In [13]:
master_trackman['PlayResult'].unique()

array(['Undefined', 'Out', 'Single', 'Double', 'FieldersChoice', 'Error',
       'Sacrifice', 'Triple', 'HomeRun', 'StolenBase', 'CaughtStealing',
       'SIngle', 'Homerun', 'FIeldersChoice', 'triple'], dtype=object)

In [30]:
## initializing new baserunner columns

for i in [1, 2, 3]:
    
    master_trackman[f"runner_on_{i}"] = None
    
## putting the batter on base depending on hit type

# iterrating through each row in the df
rows = []
runner_on_1, runner_on_2, runner_on_3 = None, None, None

for _, row in master_trackman.iterrows():
    
    hit_type, k_or_bb, batter, inning_state = row['PlayResult'], row['KorBB'], row['Batter'], row['Top/Bottom']

    # hard coding different batter scenarios
    if (hit_type == 'Single') | (hit_type == 'SIngle'):
        runner_on_1 = batter
    elif (hit_type == 'Double'):
        runner_on_2 = batter
    elif (hit_type == 'Triple') | (hit_type == 'triple'):
        runner_on_3 = batter
    elif (hit_type == 'Homerun') | (hit_type == 'HomeRun') | (hit_type == 'homerun'):
        runner_on_1, runner_on_2, runner_on_3 = None, None, None
    elif (k_or_bb == 'Walk'):
        # force logic iterrating through potential base-out states
        if runner_on_1 is None:
            runner_on_1 = batter
        elif (runner_on_1 is not None):
            runner_on_2 = runner_on_1
            runner_on_1 = batter
        elif ((runner_on_1 is not None) & (runner_on_2 is not None)):
            runner_on_3 = runner_on_2
            runner_on_2 = runner_on_1
            runner_on_1 = batter
        elif (((runner_on_1 is not None) & (runner_on_2 is not None)) & (runner_on_3 is not None)):
            runner_on_3 = runner_on_2
            runner_on_2 = runner_on_1
            runner_on_1 = batter
    elif ((hit_type == "FieldersChoice") | (hit_type == "FIeldersChoice")):
        runner_on_1 = batter
    elif (hit_type == 'StolenBase'):
        if((runner_on_1 is not None) & (runner_on_2 is None)):
            runner_on_2 = runner_on_1
            runner_on_1 = None
        ## assuming double steal when possible
        elif(((runner_on_1 is not None) & (runner_on_2 is not None))):
            runner_on_3 = runner_on_2
            runner_on_2 = runner_on_1
            runner_on_1 = None
        elif(((runner_on_1 is None) & (runner_on_2 is not None))):
            runner_on_3 = runner_on_2
            runner_on_2 = None
    elif(hit_type == 'CaughtStealing'):
        if((runner_on_1 is not None) & (runner_on_2 is None)):
            runner_on_1 = None
        ## assuming attempted double steal, lead runner out
        elif(((runner_on_1 is not None) & (runner_on_2 is not None))):
            runner_on_2 = runner_on_1
            runner_on_1 = None
        elif(((runner_on_1 is None) & (runner_on_2 is not None))):
            runner_on_2 = None
    elif(hit_type == 'Error'):
        runner_on_1 = batter
    elif(hit_type == 'Out'):
        continue
    
    row['runner_on_1'] = runner_on_1
    row['runner_on_2'] = runner_on_2
    row['runner_on_3'] = runner_on_3
    
    rows.append(row)
    
result = pd.DataFrame(rows)
print(result)
        

         PitchNo        Date         Time  PAofInning  PitchofPA  \
0              1  2025-04-08  19:04:21.64           1          1   
1              2  2025-04-08  19:04:34.59           1          2   
2              3  2025-04-08  19:04:49.95           1          3   
3              4  2025-04-08  19:05:03.44           1          4   
4              5  2025-04-08  19:05:34.89           2          1   
5              6  2025-04-08  19:06:26.75           2          2   
6              7  2025-04-08  19:06:43.81           2          3   
7              8  2025-04-08  19:07:04.44           2          4   
8              9  2025-04-08  19:07:35.69           2          5   
...          ...         ...          ...         ...        ...   
1325994      306  2025-04-04  21:25:44.99           1          3   
1325995      307  2025-04-04  21:25:59.56           1          4   
1325996      308  2025-04-04  21:26:20.31           1          5   
1325997      309  2025-04-04  21:26:36.96       

In [36]:
## if inning switches over, empty the bases
result['half_inning'] = result['Inning'].astype(str) + result['Top/Bottom']
result['new_half_inning'] = (result['half_inning'] != result['half_inning'].shift(-1))
runner_cols = ["runner_on_1", "runner_on_2", "runner_on_3"]

for col in runner_cols:
    
    result.loc[result['new_half_inning'], col] = None

In [37]:
result.head(27)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,half_inning,new_half_inning,runner_on_1st,runner_on_2nd,runner_on_3rd
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,1Top,False,None,None,None
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",8.312280e+05,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [38]:
result['runner_on_1'] = result['runner_on_1'].shift(1)
result['runner_on_2'] = result['runner_on_2'].shift(1)
result['runner_on_3'] = result['runner_on_3'].shift(1)

In [41]:
result.head(19)

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,runner_on_1,runner_on_2,runner_on_3,half_inning,new_half_inning,runner_on_1st,runner_on_2nd,runner_on_3rd
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.02806,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.62139,0.51094,5.83620,-16.09076,18.23107,4.19040,1.21203,-0.29135,83.11015,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.46398,10.48615,-0.41250,50.0,5.34285,2.64992,-130.67941,-8.01575,-4.29972,25.18165,-13.87538,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.39177,NaN,NaN,6.91791,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.0,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1.000114e+09,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.66289,-131.57487,12.59083,5.61912,-7.52234,-6.93769,-0.50945,2.80282,-2.14986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,None,None,None,1Top,False,None,None,None
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.0,Right,WAS_HUS,"Ishikawa, Keni",1.000275e+09,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.16784,1.675543,-0.415000,2782.722426,155.127014,11:15,5.53374,0.74155,4.99315,-35.98352,6.64516,-2.48776,4.11759,0.14240,75.36300,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.118